In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DSML\\Condition2Cure\\notebook'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DSML\\Condition2Cure'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_id:str
    local_data_file:Path
    unzip_dir:Path

In [5]:
from Condition2Cure.constants import *
from Condition2Cure.utils.helpers import *
from Condition2Cure.utils.execptions import *

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_id=config.source_id,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
import gdown
import zipfile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            gdown.download(id=self.config.source_id, output=self.config.local_data_file, quiet=False)
            print(f"[INFO] Downloaded file: {self.config.local_data_file}")
        else:
            print(f"[INFO] File already exists: {self.config.local_data_file} ({get_size(Path(self.config.local_data_file))})")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            print(f"[INFO] Extracted files to: {unzip_path}")


In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise CustomException(str(e), sys)

[2025-06-21 16:01:52,030: INFO: helpers: yaml file: config\config.yaml loaded successfully]
[2025-06-21 16:01:52,048: INFO: helpers: yaml file: config\params.yaml loaded successfully]
[2025-06-21 16:01:52,053: INFO: helpers: yaml file: config\schema.yaml loaded successfully]
[2025-06-21 16:01:52,056: INFO: helpers: created directory at: artifacts]
[2025-06-21 16:01:52,058: INFO: helpers: created directory at: artifacts/data_ingestion]


Downloading...
From (original): https://drive.google.com/uc?id=10dmW5TSCk91lukqWcAeBSt9pjjv-xaFW
From (redirected): https://drive.google.com/uc?id=10dmW5TSCk91lukqWcAeBSt9pjjv-xaFW&confirm=t&uuid=a2805e46-0595-4265-80f4-b1dde56daa49
To: f:\Files\DSML\Condition2Cure\artifacts\data_ingestion\data.zip
100%|██████████| 31.3M/31.3M [00:02<00:00, 11.5MB/s]


[INFO] Downloaded file: artifacts/data_ingestion/data.zip
[INFO] Extracted files to: artifacts/data_ingestion
